In [1]:
import pandas_gbq
import pickle
import pandas as pd
import numpy as np
import re
import time

# TODO: Set project_id to your Google Cloud Platform project ID.
project_id = "xxxxxx-yyyyyy-######"







In [26]:
# use this to get the table definition of individual tables in Big Query's tables

# substitute the table name you want the schema for in the "table_name = " line below


sql = """

SELECT column_name
FROM `bigquery-public-data`.stackoverflow.INFORMATION_SCHEMA.COLUMNS  
WHERE table_name = 'posts_questions' 

"""
df=pandas_gbq.read_gbq(sql, project_id=project_id)

In [27]:
df

,column_name
0,id
1,title
2,body
3,accepted_answer_id
4,answer_count
5,comment_count
6,community_owned_date
7,creation_date
8,favorite_count
9,last_activity_date


In [2]:
# Here is the code to extract the records from one table and load it into a pandas data frame

# note that this can run a long time, depending on the number of records you are trying to load


sql = """
SELECT * FROM `bigquery-public-data.stackoverflow.posts_questions` WHERE (tags LIKE '%plot%' 
OR tags LIKE '%animation%'
OR tags LIKE '%d3%'
OR tags LIKE '%graph%'
OR tags LIKE '%chart%'
OR tags LIKE '%color%'
OR tags LIKE '%3d%'
OR tags LIKE '%visualiz%'
OR tags LIKE '%maps%'
OR tags LIKE '%drawing%'
OR tags LIKE '%line%'
OR tags LIKE '%seaborn%'
OR tags LIKE '%geospatial%'
OR tags LIKE '%plyr%'
OR tags LIKE '%graphviz%'
OR tags LIKE '%diagram%')
"""



dfPostQuestionsFiltered = pandas_gbq.read_gbq(sql, project_id=project_id)

In [3]:
np.shape(dfPostQuestionsFiltered)


(692177, 20)

In [37]:
dfPostQuestionsFiltered

,Unnamed: 0,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,...,seaborn,geospatial,stata,plyr,pie-chart,graphviz,spss,diagram,qlikview,altair
0,0,7844124,Disable map controls in Google Maps embed,<p>Is it possible to disable map controls by a...,7846901.0,5,1,NaN,2011-10-21 01:35:21.823000+00:00,3.0,...,0,0,0,0,0,0,0,0,0,0
1,1,8018288,Stack Points in ggplot,<p>I am making a dotplot using <code>ggplot</c...,8018341.0,1,0,NaN,2011-11-05 05:01:34.233000+00:00,3.0,...,0,0,0,0,0,0,0,0,0,0
2,2,8227828,RGB to HSB Algorithm,<blockquote>\r\n <p><strong>Possible Duplicat...,8227891.0,2,1,NaN,2011-11-22 13:45:51.707000+00:00,4.0,...,0,0,0,0,0,0,0,0,0,0
3,3,7976438,I have a few EMF files. How I can convert them...,<p>I have few EMF (Enhanced Windows Metafile) ...,16261194.0,2,0,NaN,2011-11-02 06:08:48.397000+00:00,1.0,...,0,0,0,0,0,0,0,0,0,0
4,4,8085770,Highlight line in flot chart,<p>Is it possible to highlight a line chart wi...,11623415.0,3,2,NaN,2011-11-10 20:22:48.423000+00:00,NaN,...,0,0,0,0,0,0,0,0,0,0
5,5,7688275,save yaxis legends as a separate grob?,<p>I have a very large scatterplot of two cate...,15642634.0,1,9,NaN,2011-10-07 13:56:20.727000+00:00,4.0,...,0,0,0,0,0,0,0,0,0,0
6,6,7726367,How do I add a timestamp when digitally signin...,<p>I tried to implement in .NET C# an utility ...,7748038.0,1,3,NaN,2011-10-11 12:56:03.630000+00:00,2.0,...,0,0,0,0,0,0,0,0,0,0
7,7,7708534,"Create a thumbnail preview of documents (PDF, ...",<p>When users upload certain files to my site ...,7741591.0,2,3,NaN,2011-10-10 04:27:53.243000+00:00,4.0,...,0,0,0,0,0,0,0,0,0,0
8,8,8074161,How to show different inlines depending of cur...,<p>Given a model named <code>MainModel</code> ...,NaN,5,1,NaN,2011-11-10 02:02:25.853000+00:00,3.0,...,0,0,0,0,0,0,0,0,0,0
9,9,7907827,How can I get (and set) current bash cursor po...,<p>I have a python script that takes manages t...,NaN,1,0,NaN,2011-10-26 19:31:39.300000+00:00,5.0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
# read the model tags file to be used to create the tag features

dfModelTags = pd.read_csv("data/stackoverflow/modeltags.csv")

In [3]:
dfModelTags

,tagtext
0,javascript
1,java
2,c#
3,php
4,python
5,c++
6,node.js
7,objective-c
8,vb.net
9,scala


In [5]:
# add column to Question df to store the parsed tags
dfPostQuestionsFiltered['parsedtags'] = ''

In [87]:
# add columns to Question df to hold one-hot encoding for all model tags

for i in range(len(dfModelTags)):
    dfPostQuestionsFiltered[dfModelTags['tagtext'][i]] = 0
dfPostQuestionsFiltered['r'] = 0

In [7]:
# look for model tags in the tag field of each record
# for any model tag found in the tag field, add the model tag to the
# parsedtag field, and also set the one-hot encoded field for the model tag to 1

##  CHANGE THIS TO USE df ARRAY OPERATIONS TO IMPROVE PERFORMANCE


sttime=time.time()
for j in range(len(dfPostQuestionsFiltered)):
    tmpparsedtags=''
    for k in range(len(dfModelTags)):
        if dfModelTags['tagtext'][k] in dfPostQuestionsFiltered['tags'][j]:
            if tmpparsedtags == '':
                tmpparsedtags = dfModelTags['tagtext'][k]
            else:
                tmpparsedtags = tmpparsedtags + ' | ' + dfModelTags['tagtext'][k]
            dfPostQuestionsFiltered.loc[j, dfModelTags['tagtext'][k]] = 1
    dfPostQuestionsFiltered.loc[j, 'parsedtags'] = tmpparsedtags
    if j%10000 == 0:
        print('records processed',j,'elapsed',(time.time()-sttime)/60,'minutes')
print('total records processed',j,'total elapsed',(time.time()-sttime)/60,'minutes')

records processed 0 elapsed 0.012316560745239258 minutes
records processed 10000 elapsed 8.733399939537048 minutes
records processed 20000 elapsed 18.325248289108277 minutes
records processed 30000 elapsed 27.80505315065384 minutes
records processed 40000 elapsed 39.05173126061757 minutes
records processed 50000 elapsed 49.20510991811752 minutes
records processed 60000 elapsed 58.56901618242264 minutes
records processed 70000 elapsed 70.2740238984426 minutes
records processed 80000 elapsed 81.3103128194809 minutes
records processed 90000 elapsed 92.34226816892624 minutes
records processed 100000 elapsed 102.98463892141977 minutes
records processed 110000 elapsed 114.32539040247599 minutes
records processed 120000 elapsed 124.44644161462784 minutes
records processed 130000 elapsed 133.7144049882889 minutes
records processed 140000 elapsed 142.85684690475463 minutes
records processed 150000 elapsed 153.12582041422527 minutes
records processed 160000 elapsed 163.91671100060145 minutes
rec

In [83]:
# Here is a separate process to check for references to the r language in the question body
# and set a separate 'r' one-hot encoded column to the questions df
#
# we cannot use the same one-hot process used on the tags above, since it will almost always find occurrences of 'r'
# in the tags field that are just parts of other terms (for example, the "r" in "chart")
#
# So, here we look for standalone references to r in the question body instead
#
dfPostQuestionsFiltered['r'] = 0

def testforr(x):
    if (' r ' in x) or (' r,' in x) or (' r.' in x) or (' r:' in x):
        return 1
    else:
        return 0

sttime=time.time()

dfPostQuestionsFiltered['r'] = dfPostQuestionsFiltered['body'].map(testforr)
# dfPostQuestionsFiltered['parsedtags'] = dfPostQuestionsFiltered['parsedtags'].map(updateparsedtags)

print(time.time()-sttime)

12.314145565032959


In [81]:
dfPostQuestionsFiltered.loc[4]

Unnamed: 0                                                                  4
id                                                                    8085770
title                                            Highlight line in flot chart
body                        <p>Is it possible to highlight a line chart wi...
accepted_answer_id                                                1.16234e+07
answer_count                                                                3
comment_count                                                               2
community_owned_date                                                      NaN
creation_date                                2011-11-10 20:22:48.423000+00:00
favorite_count                                                            NaN
last_activity_date                           2014-11-17 10:22:56.853000+00:00
last_edit_date                                                            NaN
last_editor_display_name                                        

In [77]:
np.sum(dfPostQuestionsFiltered.loc[:,'javascript':'r'])

javascript        98380
java             133447
c#                46523
php               22463
python            66571
c++               15947
node.js            5286
objective-c       10589
vb.net             1992
scala              1193
matlab             8742
perl               1683
delphi             1041
matplotlib        39144
animation         53127
d3                36277
ggplot2           28131
plot             101625
graph            135722
chart             67468
highcharts        20531
vbscript            208
colors            16907
pyspark             170
dplyr             13788
f#                  314
3d                72470
sas                2874
fortran             139
maps              90986
lisp                122
julia               419
powerbi             265
drawing            6744
line             104793
plotly             5040
bar-chart          4517
visualization     15186
tableau             439
seaborn            3240
geospatial         3215
stata           

In [8]:
# write the dataframe to a tsv
dfPostQuestionsFiltered.to_csv("data/stackoverflow/PostQuestionsFiltered_V3_parsed.tsv", sep='\t')

In [4]:
# USE THIS CODE TO LOAD THE STACK OVERFLOW POST QUESTIONS TO A PANDAS DATAFRAME -- Note that you may have to change the path to the .tsv 
# file in the statement below depending on the location you downladed the .tsv to
dfPostQuestionsFiltered = pd.read_csv("data/stackoverflow/PostQuestionsFiltered_V3_parsed.tsv", sep='\t')

In [53]:
dfPostQuestionsFiltered

,Unnamed: 0,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,...,seaborn,geospatial,stata,plyr,pie-chart,graphviz,spss,diagram,qlikview,altair
0,0,7844124,Disable map controls in Google Maps embed,<p>Is it possible to disable map controls by a...,7846901.0,5,1,NaN,2011-10-21 01:35:21.823000+00:00,3.0,...,0,0,0,0,0,0,0,0,0,0
1,1,8018288,Stack Points in ggplot,<p>I am making a dotplot using <code>ggplot</c...,8018341.0,1,0,NaN,2011-11-05 05:01:34.233000+00:00,3.0,...,0,0,0,0,0,0,0,0,0,0
2,2,8227828,RGB to HSB Algorithm,<blockquote>\r\n <p><strong>Possible Duplicat...,8227891.0,2,1,NaN,2011-11-22 13:45:51.707000+00:00,4.0,...,0,0,0,0,0,0,0,0,0,0
3,3,7976438,I have a few EMF files. How I can convert them...,<p>I have few EMF (Enhanced Windows Metafile) ...,16261194.0,2,0,NaN,2011-11-02 06:08:48.397000+00:00,1.0,...,0,0,0,0,0,0,0,0,0,0
4,4,8085770,Highlight line in flot chart,<p>Is it possible to highlight a line chart wi...,11623415.0,3,2,NaN,2011-11-10 20:22:48.423000+00:00,NaN,...,0,0,0,0,0,0,0,0,0,0
5,5,7688275,save yaxis legends as a separate grob?,<p>I have a very large scatterplot of two cate...,15642634.0,1,9,NaN,2011-10-07 13:56:20.727000+00:00,4.0,...,0,0,0,0,0,0,0,0,0,0
6,6,7726367,How do I add a timestamp when digitally signin...,<p>I tried to implement in .NET C# an utility ...,7748038.0,1,3,NaN,2011-10-11 12:56:03.630000+00:00,2.0,...,0,0,0,0,0,0,0,0,0,0
7,7,7708534,"Create a thumbnail preview of documents (PDF, ...",<p>When users upload certain files to my site ...,7741591.0,2,3,NaN,2011-10-10 04:27:53.243000+00:00,4.0,...,0,0,0,0,0,0,0,0,0,0
8,8,8074161,How to show different inlines depending of cur...,<p>Given a model named <code>MainModel</code> ...,NaN,5,1,NaN,2011-11-10 02:02:25.853000+00:00,3.0,...,0,0,0,0,0,0,0,0,0,0
9,9,7907827,How can I get (and set) current bash cursor po...,<p>I have a python script that takes manages t...,NaN,1,0,NaN,2011-10-26 19:31:39.300000+00:00,5.0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# retrieving the answers that belong to the questions retrieved above

sql = """
SELECT * FROM `bigquery-public-data.stackoverflow.posts_answers` WHERE parent_id IN 
    (SELECT id FROM `bigquery-public-data.stackoverflow.posts_questions` WHERE (tags LIKE '%plot%' 
OR tags LIKE '%animation%'
OR tags LIKE '%d3%'
OR tags LIKE '%graph%'
OR tags LIKE '%chart%'
OR tags LIKE '%color%'
OR tags LIKE '%3d%'
OR tags LIKE '%visualiz%'
OR tags LIKE '%maps%'
OR tags LIKE '%drawing%'
OR tags LIKE '%line%'
OR tags LIKE '%seaborn%'
OR tags LIKE '%geospatial%'
OR tags LIKE '%plyr%'
OR tags LIKE '%graphviz%'
OR tags LIKE '%diagram%'))
"""
dfPostAnswersFiltered = pandas_gbq.read_gbq(sql, project_id=project_id)

In [12]:
np.shape(dfPostAnswersFiltered)

(889576, 20)

In [23]:
# add columns to answers to be used for modeling answers
# check for number of code snippets, images, output samples, numbered lists (like instruction steps)
# and unordered lists (bullet lists)

##  CHANGE THIS TO USE df ARRAY OPERATIONS TO IMPROVE PERFORMANCE

sttime=time.time()
for j in range(len(dfPostAnswersFiltered)):
    dfPostAnswersFiltered.loc[j, 'UnorderedLists'] = dfPostAnswersFiltered.loc[j,'body'].count('<ul>')
    dfPostAnswersFiltered.loc[j, 'OrderedLists'] = dfPostAnswersFiltered.loc[j,'body'].count('<ol>')
    dfPostAnswersFiltered.loc[j, 'CodeSnippets'] = dfPostAnswersFiltered.loc[j,'body'].count('<code>')
    dfPostAnswersFiltered.loc[j, 'SampleOutput'] = dfPostAnswersFiltered.loc[j,'body'].count('<samp>')
    dfPostAnswersFiltered.loc[j, 'Images'] = dfPostAnswersFiltered.loc[j,'body'].count('<img>')
    if j%10000 == 0:
        print('records processed',j,'elapsed',(time.time()-sttime)/60,'minutes')
print('total records processed',j,'total elapsed',(time.time()-sttime)/60,'minutes')


records processed 0 elapsed 0.04119335015614827 minutes
records processed 10000 elapsed 7.0542139410972595 minutes
records processed 20000 elapsed 14.43062025308609 minutes
records processed 30000 elapsed 22.371170830726623 minutes
records processed 40000 elapsed 29.96180255015691 minutes
records processed 50000 elapsed 37.05892446438472 minutes
records processed 60000 elapsed 44.25219342311223 minutes
records processed 70000 elapsed 51.904606036345164 minutes
records processed 80000 elapsed 59.65975027481715 minutes
records processed 90000 elapsed 67.12773571809133 minutes
records processed 100000 elapsed 74.5397210876147 minutes
records processed 110000 elapsed 82.0193485657374 minutes
records processed 120000 elapsed 89.47556643883387 minutes
records processed 130000 elapsed 96.91927777131399 minutes
records processed 140000 elapsed 104.14839432636897 minutes
records processed 150000 elapsed 111.24959344466528 minutes
records processed 160000 elapsed 118.60586909453075 minutes
recor

In [36]:
dfPostAnswersFiltered

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,...,parent_id,post_type_id,score,tags,view_count,UnorderedLists,OrderedLists,CodeSnippets,SampleOutput,Images
0,55245183,None,<p><strong>Swift 4.2</strong></p>\n\n<pre><cod...,None,None,0,NaT,2019-03-19 15:57:46.060000+00:00,None,2019-03-19 15:57:46.060000+00:00,...,27728466,2,0,None,None,0.0,0.0,1.0,0.0,0.0
1,55245207,None,<p>Specify the <code>ax</code> argument to <a ...,None,None,0,NaT,2019-03-19 15:58:34.173000+00:00,None,2019-03-19 15:58:34.173000+00:00,...,23876588,2,0,None,None,0.0,0.0,3.0,0.0,0.0
2,55245381,None,"<p>Ok, I googled for some more time and founde...",None,None,0,NaT,2019-03-19 16:07:04.040000+00:00,None,2019-03-19 16:07:04.040000+00:00,...,55244349,2,0,None,None,0.0,0.0,0.0,0.0,0.0
3,55245456,None,<p>Yikes! I forgot to set the GameObject to tr...,None,None,0,NaT,2019-03-19 16:11:46.770000+00:00,None,2019-03-19 16:11:46.770000+00:00,...,55244515,2,0,None,None,0.0,0.0,0.0,0.0,0.0
4,55245503,None,"<p>I decided to use the <a href=""https://docs....",None,None,0,NaT,2019-03-19 16:13:42.220000+00:00,None,2019-03-19 16:13:42.220000+00:00,...,55127820,2,0,None,None,0.0,0.0,0.0,0.0,0.0
5,55245748,None,"<p>Ok sorry that wasn't clear. The ""solution"" ...",None,None,0,NaT,2019-03-19 16:26:56.300000+00:00,None,2019-03-19 16:26:56.300000+00:00,...,55229446,2,0,None,None,0.0,0.0,1.0,0.0,0.0
6,55245847,None,"<p>Based on my experience, I suggest you creat...",None,None,1,NaT,2019-03-19 16:33:17.213000+00:00,None,2019-03-19 16:33:17.213000+00:00,...,55245250,2,0,None,None,0.0,0.0,0.0,0.0,0.0
7,55245995,None,"<p>You should add a parameter <strong>""out_fil...",None,None,1,NaT,2019-03-19 16:40:45.230000+00:00,None,2019-03-19 16:40:45.230000+00:00,...,54243959,2,0,None,None,0.0,0.0,0.0,0.0,0.0
8,55246054,None,<p>Here is a quick sample solution.</p>\n\n<ul...,None,None,0,NaT,2019-03-19 16:43:38.093000+00:00,None,2019-03-19 16:43:38.093000+00:00,...,55245566,2,0,None,None,1.0,0.0,7.0,0.0,0.0
9,55246085,None,<p>This happens if the designer applies a post...,None,None,0,NaT,2019-03-19 16:45:09.013000+00:00,None,2019-03-19 16:45:09.013000+00:00,...,55157762,2,0,None,None,0.0,0.0,0.0,0.0,0.0


In [35]:
np.sum(dfPostAnswersFiltered.loc[:,'UnorderedLists':'Images'])

UnorderedLists      43479.0
OrderedLists        29071.0
CodeSnippets      2133476.0
SampleOutput            0.0
Images                  0.0
dtype: float64

In [24]:
# write the dataframe to a tsv
dfPostAnswersFiltered.to_csv("data/stackoverflow/PostAnswersFiltered_V3_parsed.tsv", sep='\t')

In [4]:
# USE THIS CODE TO LOAD THE STACK OVERFLOW POST ANSWERS TO A PANDAS DATAFRAME -- Note that you may have to change the path to the .tsv 
# file in the statement below depending on the location you downladed the .tsv to
dfPostAnswersFiltered = pd.read_csv("data/stackoverflow/PostAnswersFiltered_V3_parsed.tsv", sep='\t')

In [14]:
dfPostAnswersFiltered

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,55245183,None,<p><strong>Swift 4.2</strong></p>\n\n<pre><cod...,None,None,0,NaT,2019-03-19 15:57:46.060000+00:00,None,2019-03-19 15:57:46.060000+00:00,NaT,None,NaN,None,5469908.0,27728466,2,0,None,None
1,55245207,None,<p>Specify the <code>ax</code> argument to <a ...,None,None,0,NaT,2019-03-19 15:58:34.173000+00:00,None,2019-03-19 15:58:34.173000+00:00,NaT,None,NaN,None,846924.0,23876588,2,0,None,None
2,55245381,None,"<p>Ok, I googled for some more time and founde...",None,None,0,NaT,2019-03-19 16:07:04.040000+00:00,None,2019-03-19 16:07:04.040000+00:00,NaT,None,NaN,None,11227234.0,55244349,2,0,None,None
3,55245456,None,<p>Yikes! I forgot to set the GameObject to tr...,None,None,0,NaT,2019-03-19 16:11:46.770000+00:00,None,2019-03-19 16:11:46.770000+00:00,NaT,None,NaN,None,2872160.0,55244515,2,0,None,None
4,55245503,None,"<p>I decided to use the <a href=""https://docs....",None,None,0,NaT,2019-03-19 16:13:42.220000+00:00,None,2019-03-19 16:13:42.220000+00:00,NaT,None,NaN,None,8302617.0,55127820,2,0,None,None
5,55245748,None,"<p>Ok sorry that wasn't clear. The ""solution"" ...",None,None,0,NaT,2019-03-19 16:26:56.300000+00:00,None,2019-03-19 16:26:56.300000+00:00,NaT,None,NaN,None,3047583.0,55229446,2,0,None,None
6,55245847,None,"<p>Based on my experience, I suggest you creat...",None,None,1,NaT,2019-03-19 16:33:17.213000+00:00,None,2019-03-19 16:33:17.213000+00:00,NaT,None,NaN,None,10122536.0,55245250,2,0,None,None
7,55245995,None,"<p>You should add a parameter <strong>""out_fil...",None,None,1,NaT,2019-03-19 16:40:45.230000+00:00,None,2019-03-19 16:40:45.230000+00:00,NaT,None,NaN,None,10142264.0,54243959,2,0,None,None
8,55246054,None,<p>Here is a quick sample solution.</p>\n\n<ul...,None,None,0,NaT,2019-03-19 16:43:38.093000+00:00,None,2019-03-19 16:43:38.093000+00:00,NaT,None,NaN,None,11185338.0,55245566,2,0,None,None
9,55246085,None,<p>This happens if the designer applies a post...,None,None,0,NaT,2019-03-19 16:45:09.013000+00:00,None,2019-03-19 16:45:09.013000+00:00,NaT,None,NaN,None,1851191.0,55157762,2,0,None,None


In [48]:
# retrieving user ids for owners of the answers retrieved above

sql = """
SELECT * FROM `bigquery-public-data.stackoverflow.users` WHERE id IN
    (SELECT owner_user_id FROM `bigquery-public-data.stackoverflow.posts_answers` WHERE parent_id IN 
        (SELECT id FROM `bigquery-public-data.stackoverflow.posts_questions` WHERE (tags LIKE '%plot%' 
            OR tags LIKE '%animation%'
            OR tags LIKE '%d3%'
            OR tags LIKE '%graph%'
            OR tags LIKE '%chart%'
            OR tags LIKE '%color%'
            OR tags LIKE '%3d%'
            OR tags LIKE '%visualiz%'
            OR tags LIKE '%maps%'
            OR tags LIKE '%drawing%'
            OR tags LIKE '%line%'
            OR tags LIKE '%seaborn%'
            OR tags LIKE '%geospatial%'
            OR tags LIKE '%plyr%'
            OR tags LIKE '%graphviz%'
            OR tags LIKE '%diagram%')))
"""
dfUsersFiltered = pandas_gbq.read_gbq(sql, project_id=project_id)

In [49]:
np.shape(dfUsersFiltered)

(304366, 13)

In [50]:
dfUsersFiltered

,id,display_name,about_me,age,creation_date,last_access_date,location,reputation,up_votes,down_votes,views,profile_image_url,website_url
0,1228,Will,<p>Downvoted a post? Want to let the author kn...,None,2008-08-13 13:58:55.613000+00:00,2019-08-30 19:32:14.180000+00:00,United States,114479,4072,8499,61068,None,https://idownvotedbecau.se
1,2009,hometoast,<p>(your about me is currently blank)</p>,None,2008-08-19 19:16:03.210000+00:00,2019-08-29 14:39:17.043000+00:00,"Mountain Top, PA",9604,1400,72,892,None,http://devtoast.com
2,2684,Martin Marconcini,<p>I do Android development with Kotlin/Java a...,None,2008-08-24 14:38:27.090000+00:00,2019-08-31 20:36:33.830000+00:00,"Almere, NL",17343,2555,159,5171,https://i.stack.imgur.com/Z9oZ0.jpg?s=128&g=1,https://professorneurus.wordpress.com
3,5049,macbirdie,"<p>Freelancer, doing primarily iOS development...",None,2008-09-07 15:03:23.483000+00:00,2019-08-27 14:14:57.663000+00:00,"Poznan, Poland",14626,344,49,3603,None,https://macbirdie.net
4,6651,Alex Angas,None,None,2008-09-15 12:25:40.173000+00:00,2019-08-30 16:49:12.890000+00:00,"London, United Kingdom",35087,3339,271,3365,None,https://alexangas.com
5,8562,mdb,None,None,2008-09-15 16:20:52.917000+00:00,2017-11-14 13:45:05.013000+00:00,Netherlands,45600,1084,65,1841,None,None
6,8912,Ted Naleid,None,None,2008-09-15 17:12:19.533000+00:00,2019-08-29 13:28:19.343000+00:00,United States,22733,1251,26,1292,None,http://naleid.com/
7,9936,Martin Cote,<p>Graphics Engineer at Unity3D.</p>,None,2008-09-15 20:30:09.543000+00:00,2019-08-23 18:37:35.430000+00:00,"Montreal, Canada",22119,316,72,1305,None,None
8,13820,Sridhar Iyer,Wired and Weird,None,2008-09-16 20:57:40.933000+00:00,2019-08-30 19:03:38.590000+00:00,"Santa Clara, CA",2021,176,22,687,None,http://sridharv.net
9,17413,Nifle,<p><code>A rather lazy sysadmin and sometimes ...,None,2008-09-18 08:34:03.827000+00:00,2019-08-30 12:50:59.610000+00:00,Sweden,8624,2964,422,1470,None,http://www.kiva.org/


In [51]:
# write the dataframe to a tsv
dfUsersFiltered.to_csv("data/stackoverflow/UsersFiltered_V3.tsv", sep='\t')

In [4]:
# USE THIS CODE TO LOAD THE STACK OVERFLOW USERS FOR ABOVE ANSWERS TO A PANDAS DATAFRAME -- Note that you may have to change the path to the .tsv 
# file in the statement below depending on the location you downladed the .tsv to
dfUsersFiltered = pd.read_csv("data/stackoverflow/UsersFiltered_V3.tsv", sep='\t')

In [45]:
# retrieving badges for users who provided the answers retrieved above

sql = """
SELECT * FROM `bigquery-public-data.stackoverflow.badges` WHERE user_id IN
    (SELECT id FROM `bigquery-public-data.stackoverflow.users` WHERE id IN
        (SELECT owner_user_id FROM `bigquery-public-data.stackoverflow.posts_answers` WHERE parent_id IN 
            (SELECT id FROM `bigquery-public-data.stackoverflow.posts_questions` WHERE (tags LIKE '%plot%' 
                OR tags LIKE '%animation%'
                OR tags LIKE '%d3%'
                OR tags LIKE '%graph%'
                OR tags LIKE '%chart%'
                OR tags LIKE '%color%'
                OR tags LIKE '%3d%'
                OR tags LIKE '%visualiz%'
                OR tags LIKE '%maps%'
                OR tags LIKE '%drawing%'
                OR tags LIKE '%line%'
                OR tags LIKE '%seaborn%'
                OR tags LIKE '%geospatial%'
                OR tags LIKE '%plyr%'
                OR tags LIKE '%graphviz%'
                OR tags LIKE '%diagram%'))))
"""
dfUserBadgesFiltered = pandas_gbq.read_gbq(sql, project_id=project_id)

In [46]:
np.shape(dfUserBadgesFiltered)

(10162582, 6)

In [47]:
dfUserBadgesFiltered

,id,name,date,user_id,class,tag_based
0,262290,Cleanup,2009-03-18 14:17:32.233000+00:00,6430,3,False
1,293628,c#,2009-04-25 18:40:46.630000+00:00,65358,2,True
2,422453,Beta,2008-09-16 00:00:00+00:00,287,2,False
3,627811,Mortarboard,2010-01-19 21:37:37.807000+00:00,96168,3,False
4,910116,Promoter,2010-07-09 09:52:05.647000+00:00,50122,3,False
5,912160,Promoter,2010-07-09 09:54:35.937000+00:00,4642,3,False
6,914907,Benefactor,2010-07-09 09:55:47.547000+00:00,18192,3,False
7,962093,Taxonomist,2010-07-31 08:43:48.383000+00:00,386102,2,False
8,1033990,c,2010-09-04 19:34:32.380000+00:00,68204,2,True
9,1039977,git,2010-07-04 07:19:15.717000+00:00,193688,3,True


In [52]:
# write the Badges dataframe to a tsv
dfUserBadgesFiltered.to_csv("data/stackoverflow/UserBadgesFiltered_V3.tsv", sep='\t')

In [4]:
# USE THIS CODE TO LOAD THE STACK OVERFLOW USER BADGES TO A PANDAS DATAFRAME -- Note that you may have to change the path to the .tsv 
# file in the statement below depending on the location you downladed the .tsv to
dfUserBadgesFiltered = pd.read_csv("data/stackoverflow/UserBadgesFiltered_V3.tsv", sep='\t')

In [15]:
# retrieving the comments that belong to the questions retrieved above

sql = """
SELECT * FROM `bigquery-public-data.stackoverflow.comments` WHERE post_id IN 
    (SELECT id FROM `bigquery-public-data.stackoverflow.posts_questions` WHERE (tags LIKE '%plot%' 
OR tags LIKE '%animation%'
OR tags LIKE '%d3%'
OR tags LIKE '%graph%'
OR tags LIKE '%chart%'
OR tags LIKE '%color%'
OR tags LIKE '%3d%'
OR tags LIKE '%visualiz%'
OR tags LIKE '%maps%'
OR tags LIKE '%drawing%'
OR tags LIKE '%line%'
OR tags LIKE '%seaborn%'
OR tags LIKE '%geospatial%'
OR tags LIKE '%plyr%'
OR tags LIKE '%graphviz%'
OR tags LIKE '%diagram%'))
"""
dfCommentsFiltered = pandas_gbq.read_gbq(sql, project_id=project_id)

In [16]:
np.shape(dfCommentsFiltered)

(1184056, 7)

In [17]:
dfCommentsFiltered

,id,text,creation_date,post_id,user_id,user_display_name,score
0,41452359,SOLVED! Check this https://github.com/ikimuhen...,2014-10-16 09:50:36.890000+00:00,26350205,1833505.0,None,4
1,41453409,You also get this problem when you restore a b...,2014-10-16 10:23:14.340000+00:00,2043382,10245.0,None,4
2,41464240,"Do you think `-12.80010128657071, 1121.2874782...",2014-10-16 15:09:57.510000+00:00,22773651,1238965.0,None,4
3,41472601,Pretty much all encryption algorithms are goin...,2014-10-16 19:27:46.807000+00:00,26412089,869736.0,None,4
4,41473301,Surely F(S) = 9999 is not what you're looking ...,2014-10-16 19:48:05.153000+00:00,26412312,2144669.0,None,4
5,41474727,"""We are not allowed to do any changes with res...",2014-10-16 20:32:31.033000+00:00,26413343,1274820.0,None,4
6,41490904,I think that kind of question is better to ask...,2014-10-17 10:13:03.323000+00:00,26419181,1451635.0,None,4
7,41499177,The jqChart plugin isn't loaded... check your ...,2014-10-17 14:23:53.250000+00:00,26427123,1686485.0,None,5
8,41503605,Hmmm... it looks like you are cancelling the e...,2014-10-17 16:22:45.360000+00:00,26429055,2718864.0,None,5
9,41568509,This question appears to be off-topic because ...,2014-10-20 12:06:44.173000+00:00,26462355,47961.0,None,4


In [18]:
# write the dataframe to a tsv
dfCommentsFiltered.to_csv("data/stackoverflow/CommentsFiltered_v3.tsv", sep='\t')

In [41]:
# USE THIS CODE TO LOAD THE STACK OVERFLOW POST COMMENTS TO A PANDAS DATAFRAME -- Note that you may have to change the path to the .tsv 
# file in the statement below depending on the location you downladed the .tsv to
dfCommentsFiltered = pd.read_csv("data/stackoverflow/CommentsFiltered.tsv", sep='\t')

In [42]:
dfCommentsFiltered

,Unnamed: 0,id,text,creation_date,post_id,user_id,user_display_name,score
0,0,13959910,Don't worry about extra objects unless there i...,2012-05-24 19:03:24.103000+00:00,10743622,NaN,user166390,6
1,1,14116297,"If you're going to use NodeJS, you might as we...",2012-05-31 20:28:18.247000+00:00,10840689,NaN,user1106925,5
2,2,14834466,"WTF, you want to write your own virtual filesy...",2012-07-01 04:15:00.443000+00:00,11279477,NaN,user405725,6
3,3,13253544,"Sorry about the semicolons, I did put them in ...",2012-04-24 13:46:43.277000+00:00,10298983,NaN,user818700,0
4,4,13254271,"@Pointy - I'm so sorry, it's obviously too lat...",2012-04-24 14:11:36.220000+00:00,10298983,NaN,user818700,0
5,5,13255022,Ahhhh.. Now I see. Thank you very much. I real...,2012-04-24 14:37:07.523000+00:00,10298983,NaN,user818700,0
6,6,13256213,Here's a complete example: http://codepad.org/...,2012-04-24 15:18:30.447000+00:00,10290177,NaN,user393964,0
7,7,13279186,"I'm certainly surprised by this, and it does't...",2012-04-25 12:52:39.417000+00:00,10315746,NaN,user24359,0
8,8,13279391,"Er, I mean `Payments`",2012-04-25 12:59:19.757000+00:00,10315746,NaN,user24359,0
9,9,13283132,@dasblinkenlight great thanks. I guess this is...,2012-04-25 15:09:25.203000+00:00,10318302,NaN,user517491,0
